# Hackathon

In [ ]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
from prophet import Prophet


In [ ]:
df = pd.read_csv('train/train_data.csv')
df["MEASURED_IN_DATE"] = pd.to_datetime(df["MEASURED_IN_DATE"])
df.info()

In [ ]:
# rename cols for prophet
df.rename(columns={"MEASURED_IN_DATE": "ds", "LIQ_RATE": "y"}, inplace=True)

In [ ]:
def stan_init(m):
    """Retrieve parameters from a trained model.

    Retrieve parameters from a trained model in the format
    used to initialize a new Stan model.

    Parameters
    ----------
    m: A trained model of the Prophet class.

    Returns
    -------
    A Dictionary containing retrieved parameters of m.

    """
    res = {}
    for pname in ['k', 'm', 'sigma_obs']:
        res[pname] = m.params[pname][0]
    for pname in ['delta', 'beta']:
        res[pname] = m.params[pname][0][0]
    return res


In [ ]:
# final dataframe for big image 
test = pd.DataFrame()

# big image settings
figure = plt.gcf()
figure.set_size_inches(30, 2)

m1 = Prophet()
# fit every well separately
for i in range(len(df['WELL_NAME'].unique())):
    dff = pd.DataFrame(df.loc[df['WELL_NAME']==df['WELL_NAME'].unique()[i]])
    m2 = Prophet()

    test_data_size = int(len(dff)*0.2)
    if i>0:
        m2.fit(df[:test_data_size], init = stan_init(m1)) # todo fix crash???
    else:
        m2.fit(df[:test_data_size])
    future = m2.make_future_dataframe(periods=test_data_size) #forecasting 365 days in future
    forecast = m2.predict(future)

    m1 = m2

    test = pd.DataFrame(pd.concat([test,forecast], ignore_index=True))

In [ ]:
plt.plot(df['y'], linewidth = 0.2)
plt.plot(test['trend'], linewidth = 0.2, linestyle = '--')
plt.savefig("myplot.png", dpi = 1000)

Next i try to fit prophet with all data and then forecast something, which didn't work

In [ ]:
train_stop_name = df['WELL_NAME'][len(df)-int(len(df)*0.2)]
train_stop_index = df.index[df['WELL_NAME'] == train_stop_name][0]
train_stop_index

In [ ]:
test_data_size = len(df)-train_stop_index
test_data_size

In [ ]:
m = Prophet()
m.fit(df[:-test_data_size])

In [ ]:
future = m.make_future_dataframe(periods = 120)
forecast = m.predict(future)

In [ ]:
forecast

In [ ]:
#plt.plot(forecast["ds"].values, forecast["trend"].values)
n=2000
figure = plt.gcf()
figure.set_size_inches(16, 9)
grouped = df.groupby("WELL_NAME")
for name in df.WELL_NAME.unique():
    plt.plot(grouped.get_group(name)["y"])
#np.datetime64(df["ds"].values[0])
#plt.show()